In [1]:
import numpy as np
import pandas as pd

In [2]:
path='../data/'
list_path= path+'driver_imgs_list.csv'
print(list_path)
image_dataset = pd.read_csv(list_path)

../data/driver_imgs_list.csv


In [3]:
image_dataset.head()


,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [4]:
x=image_dataset['img']
y=image_dataset['classname']

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.20,train_size =0.80)

In [9]:
type(x_train)
x_train.size

17939

In [11]:
import os
import shutil
for i in range(0,x_train.size):
    cr_path ='../train/' + y_train.iloc[i] + '/'
    if (not os.path.exists(cr_path)):
        os.makedirs(cr_path)
    shutil.copyfile('../data/imgs/train/' + y_train.iloc[i] + '/' + x_train.iloc[i], '../train/' + y_train.iloc[i] + '/' + x_train.iloc[i])


In [12]:
for i in range(0,x_val.size):
    cr_path ='../validation/' + y_val.iloc[i] + '/'
    if (not os.path.exists(cr_path)):
        os.makedirs(cr_path)
    shutil.copyfile('../data/imgs/train/' + y_val.iloc[i] + '/' + x_val.iloc[i], '../validation/' + y_val.iloc[i] + '/' + x_val.iloc[i])


In [13]:
from keras.preprocessing.image import ImageDataGenerator
image_data_generator=ImageDataGenerator(
                        rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)

/Users/manishashivshette/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
train_generator = image_data_generator.flow_from_directory(
        '../train',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')

Found 17939 images belonging to 10 classes.


In [16]:
test_datagenerator = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagenerator.flow_from_directory(
        '../validation',
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')

Found 4485 images belonging to 10 classes.


In [17]:
from keras.models import Sequential
from keras import regularizers, optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [19]:
steps_per_epoch_array = [100,200,300,1000,2000]
epochs_array = [10,20,30,40,50]
validation_steps_array=[100,200,300,400,800]
model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch_array[4],
        epochs=epochs_array[0],
        validation_data=validation_generator,
        validation_steps=validation_steps_array[4])

Epoch 1/10
2000/2000 [==============================] - 537s 269ms/step - loss: 1.3529 - acc: 0.5275 - val_loss: 0.7079 - val_acc: 0.7908
Epoch 2/10
2000/2000 [==============================] - 548s 274ms/step - loss: 0.8334 - acc: 0.7190 - val_loss: 0.3870 - val_acc: 0.8985
Epoch 3/10
2000/2000 [==============================] - 499s 249ms/step - loss: 0.5986 - acc: 0.8018 - val_loss: 0.2718 - val_acc: 0.9263
Epoch 4/10
2000/2000 [==============================] - 517s 259ms/step - loss: 0.4686 - acc: 0.8489 - val_loss: 0.2198 - val_acc: 0.9420
Epoch 5/10
2000/2000 [==============================] - 500s 250ms/step - loss: 0.3908 - acc: 0.8760 - val_loss: 0.1781 - val_acc: 0.9565
Epoch 6/10
2000/2000 [==============================] - 482s 241ms/step - loss: 0.3389 - acc: 0.8940 - val_loss: 0.1644 - val_acc: 0.9553
Epoch 7/10
2000/2000 [==============================] - 458s 229ms/step - loss: 0.3034 - acc: 0.9047 - val_loss: 0.1394 - val_acc: 0.9636
Epoch 8/10
2000/2000 [============

In [20]:
model.metrics_names
result=model.evaluate_generator(generator=validation_generator,steps=4485)

In [21]:
print("Model loss is %f accuracy is %f "%(result[0],result[1]))

Model loss is 0.106529 accuracy is 0.973419 


In [22]:
from keras.models import load_model

model.save('model_using_keras.h5')
